In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import ttk
from numba import jit

In [ ]:
def mandelbrot(c, max_iter):
    """Compute the color of a Mandelbrot set point."""
    z = 0
    n = 0
    while abs(z) <= 2 and n < max_iter:
        z = z * z + c
        n += 1
    if n == max_iter:
        return max_iter
    return n + 1 - np.log(np.log2(abs(z)))


# Image size (pixels)
width, height = 800, 800

# Plot window
re_min, re_max = -2.0, 1.0
im_min, im_max = -1.5, 1.5

# Generate a grid of complex numbers
real = np.linspace(re_min, re_max, width)
imag = np.linspace(im_min, im_max, height)
real_grid, imag_grid = np.meshgrid(real, imag)
c = real_grid + 1j * imag_grid

# Compute the Mandelbrot image
max_iter = 256
mandelbrot_image = np.vectorize(mandelbrot)(c, max_iter)

# Plotting the Mandelbrot set
plt.figure(figsize=(10, 10))
plt.imshow(mandelbrot_image, extent=[re_min, re_max, im_min, im_max], cmap='hot')
plt.colorbar()
plt.title('Mandelbrot Set')
plt.xlabel('Re')
plt.ylabel('Im')
plt.show()


In [ ]:
@jit(nopython=True)
def mandelbrot(c, max_iter):
    z = 0.0j
    for n in range(max_iter):
        if abs(z) > 2.0:
            return n + 1 - np.log(np.log2(abs(z)))
        z = z * z + c
    return max_iter

class MandelbrotViewer(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Mandelbrot Set Viewer")
        self.geometry("800x900")  # Adjusted to accommodate progress bar

        self.fig, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.fig, master=self)
        self.plot_widget = self.canvas.get_tk_widget()
        self.plot_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Setup progress bar
        self.progress = ttk.Progressbar(self, orient="horizontal", length=300, mode="determinate")
        self.progress.pack(side=tk.TOP, pady=20)

        self.ax.set_xlabel('Re')
        self.ax.set_ylabel('Im')

        self.max_iter = 256
        self.draw_mandelbrot(-2.0, 1.0, -1.5, 1.5)

        self.fig.canvas.mpl_connect('button_press_event', self.on_press)
        self.fig.canvas.mpl_connect('button_release_event', self.on_release)

    def draw_mandelbrot(self, re_min, re_max, im_min, im_max):
        width, height = 800, 800
        real = np.linspace(re_min, re_max, width)
        imag = np.linspace(im_min, im_max, height)
        mandelbrot_image = np.zeros((width, height))

        self.progress["value"] = 0
        self.progress["maximum"] = width

        for x in range(width):
            for y in range(height):
                c = real[x] + 1j * imag[y]
                mandelbrot_image[x, y] = mandelbrot(c, self.max_iter)
            self.progress["value"] = x
            self.progress.update()  # Update progress bar for each column calculated

        self.ax.imshow(mandelbrot_image.T, extent=[re_min, re_max, im_min, im_max], origin='lower', cmap='hot')
        self.ax.set_xlim([re_min, re_max])
        self.ax.set_ylim([im_min, im_max])
        self.canvas.draw()


    def on_press(self, event):
        # Ensure that the mouse press is within the plot area
        if event.xdata is not None and event.ydata is not None:
            self.zoom_start = (event.xdata, event.ydata)
        else:
            self.zoom_start = None


    def on_release(self, event):
        # Ensure that both press and release events are within the plot area
        if self.zoom_start is not None and event.xdata is not None and event.ydata is not None:
            zoom_end = (event.xdata, event.ydata)
            if zoom_end != self.zoom_start:
                xmin, xmax = sorted([self.zoom_start[0], zoom_end[0]])
                ymin, ymax = sorted([self.zoom_start[1], zoom_end[1]])
                self.ax.clear()
                self.draw_mandelbrot(xmin, xmax, ymin, ymax)
        self.zoom_start = None  # Reset zoom_start to ensure it's always clean


if __name__ == "__main__":
    app = MandelbrotViewer()
    app.mainloop()